In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
# import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime
from bs4 import BeautifulSoup
from sklearn.model_selection import GridSearchCV


In [3]:
df = pd.read_csv(r'F:\mpst_full_data.csv', delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 14828 rows and 6 columns


In [4]:
df.head(5)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import preprocessing

Using TensorFlow backend.


In [6]:
# df=df.drop(columns=["split"])

In [7]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split',
       'synopsis_source'],
      dtype='object')

In [8]:
# df['combined']=df['title']+". "+ df['plot_synopsis']

In [9]:
type(Sequential()) == Sequential


True

In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Convolution1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import plotly.plotly as py
# import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot

In [11]:
def gdd(x):
    a=x.split()
    return len(a)

In [12]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [14]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in df['plot_synopsis'].values:
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())

In [15]:
df['preprocessed_plots']=preprocessed_synopsis

In [16]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split', 'synopsis_source',
       'preprocessed_plots'],
      dtype='object')

In [17]:
def remove_spaces(x):
    x=x.split(",")
    nospace=[]
    for item in x:
        item=item.lstrip()
        nospace.append(item)
    return (",").join(nospace)  

In [18]:
df['tags']=df['tags'].apply(remove_spaces)

In [19]:
train=df.loc[df.split=='train']
cv=df.loc[df.split=="val"]
cv=cv.reset_index()
train=train.reset_index()
test=df.loc[df.split=='test']
test=test.reset_index()

In [20]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['tags']).toarray()
y_test=vectorizer.transform(test['tags']).toarray()
y_cv=vectorizer.transform(cv['tags']).toarray()

In [21]:
import tensorflow as tf

session_conf = tf.ConfigProto(
 intra_op_parallelism_threads=1, 
                        inter_op_parallelism_threads=1, 
                      )

In [22]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split', 'synopsis_source',
       'preprocessed_plots'],
      dtype='object')

In [23]:
max(df['plot_synopsis'].apply(gdd))

11406

In [24]:
vect=Tokenizer()
vect.fit_on_texts(train['plot_synopsis'])
vocab_size = len(vect.word_index) + 1

In [25]:
vocab_size

122195

In [26]:
encoded_docs_train = vect.texts_to_sequences(train['preprocessed_plots'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1200, padding='post')
print(padded_docs_train)

[[  779  4660 62208 ...     0     0     0]
 [   51  4481   143 ...     0     0     0]
 [ 3063   429   188 ...    75   140  6946]
 ...
 [  140  2717   539 ...     0     0     0]
 [ 5118  2731  3015 ...     0     0     0]
 [ 1269  2392  2530 ...     0     0     0]]


In [27]:
encoded_docs_test = vect.texts_to_sequences(test['preprocessed_plots'])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200, padding='post')
encoded_docs_cv = vect.texts_to_sequences(cv['preprocessed_plots'])
padded_docs_cv = pad_sequences(encoded_docs_cv, maxlen=1200, padding='post')



In [28]:
session_conf = tf.ConfigProto(
 intra_op_parallelism_threads=1, 
                        inter_op_parallelism_threads=1, 
                      )

In [29]:
def f1micro(y_true, y_pred):
    return tf.py_func(f1_score(y_true, y_pred,average='mirco'),tf.double)


In [30]:
from keras.layers import Dense, Flatten, LSTM, Conv1D, Conv2D, MaxPooling1D, Dropout, Activation,GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import Adam
from time import time
from keras.callbacks import TensorBoard
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam

from keras.layers.embeddings import Embedding
from keras.callbacks import ReduceLROnPlateau

In [31]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')

In [38]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(71, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1200, 50)          4939100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1200, 50)          0         
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 71)                3621      
Total params: 4,942,721
Trainable params: 4,942,721
Non-trainable params: 0
_________________________________________________________________


In [39]:
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1micro])

model.compile(loss='binary_crossentropy', optimizer='adam')
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-simple1.hdf5', save_best_only=True)
]

history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)
# history=model.fit(padded_docs_train, y_train,validation_data=(padded_docs_cv,y_cv), epochs =5,batch_size=32,callbacks=[tensorboard])

Train on 8540 samples, validate on 949 samples
Epoch 1/5
8540/8540 [==============================] - ETA: 7:30 - loss: 0.700 - ETA: 4:11 - loss: 0.699 - ETA: 3:04 - loss: 0.698 - ETA: 2:31 - loss: 0.697 - ETA: 2:11 - loss: 0.696 - ETA: 1:58 - loss: 0.695 - ETA: 1:48 - loss: 0.694 - ETA: 1:41 - loss: 0.693 - ETA: 1:35 - loss: 0.692 - ETA: 1:31 - loss: 0.691 - ETA: 1:28 - loss: 0.690 - ETA: 1:24 - loss: 0.689 - ETA: 1:22 - loss: 0.688 - ETA: 1:20 - loss: 0.688 - ETA: 1:18 - loss: 0.687 - ETA: 1:16 - loss: 0.686 - ETA: 1:15 - loss: 0.685 - ETA: 1:13 - loss: 0.684 - ETA: 1:12 - loss: 0.683 - ETA: 1:11 - loss: 0.682 - ETA: 1:10 - loss: 0.681 - ETA: 1:09 - loss: 0.680 - ETA: 1:08 - loss: 0.678 - ETA: 1:07 - loss: 0.677 - ETA: 1:06 - loss: 0.676 - ETA: 1:06 - loss: 0.675 - ETA: 1:05 - loss: 0.674 - ETA: 1:04 - loss: 0.673 - ETA: 1:03 - loss: 0.672 - ETA: 1:03 - loss: 0.671 - ETA: 1:02 - loss: 0.670 - ETA: 1:01 - loss: 0.669 - ETA: 1:01 - loss: 0.668 - ETA: 1:00 - loss: 0.667 - ETA: 59s - los

8540/8540 [==============================] - ETA: 57s - loss: 0.16 - ETA: 54s - loss: 0.15 - ETA: 53s - loss: 0.16 - ETA: 53s - loss: 0.16 - ETA: 54s - loss: 0.15 - ETA: 54s - loss: 0.15 - ETA: 54s - loss: 0.15 - ETA: 53s - loss: 0.15 - ETA: 53s - loss: 0.15 - ETA: 53s - loss: 0.15 - ETA: 53s - loss: 0.15 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 46s - loss: 0.

8540/8540 [==============================] - ETA: 54s - loss: 0.16 - ETA: 53s - loss: 0.14 - ETA: 53s - loss: 0.15 - ETA: 52s - loss: 0.15 - ETA: 52s - loss: 0.15 - ETA: 52s - loss: 0.15 - ETA: 53s - loss: 0.15 - ETA: 53s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 52s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 51s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 50s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 49s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 48s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 47s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.14 - ETA: 46s - loss: 0.

In [32]:
predictions=model.predict([padded_docs_test])
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]


In [41]:
for val in thresholds:
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Micro-average quality numbers
Precision: 0.2023, Recall: 0.5908, F1-measure: 0.3014
Micro-average quality numbers
Precision: 0.2645, Recall: 0.4459, F1-measure: 0.3320
Micro-average quality numbers
Precision: 0.3940, Recall: 0.2172, F1-measure: 0.2801
Micro-average quality numbers
Precision: 0.5717, Recall: 0.1070, F1-measure: 0.1802
Micro-average quality numbers
Precision: 0.6348, Recall: 0.0243, F1-measure: 0.0468


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [42]:
filter_length = 300


# model = Sequential()
# model.add(Embedding(6119, 71, input_length=1200))
# model.add(Dropout(0.1))
# model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
# model.add(GlobalMaxPool1D())
# model.add(Dense(82))
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(71, activation='sigmoid'))

# model.summary()


model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-2.h5', save_best_only=True)
# ]
history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1200, 50)          4939100   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1200, 50)          0         
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 71)                3621      
_________________________________________________________________
activation_1 (Activation)    (None, 71)                0         
Total params: 4,942,721
Trainable params: 4,942,721
Non-trainable params: 0
_________________________________________________________________
Train on 8540 samples, validate on 949 samples
Epoch 1/5
8540/8540 [==============================] - ETA: 4:59 - loss: 0.952 - ETA:

8540/8540 [==============================] - ETA: 56s - loss: 0.69 - ETA: 56s - loss: 0.69 - ETA: 55s - loss: 0.69 - ETA: 55s - loss: 0.69 - ETA: 55s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.

8540/8540 [==============================] - ETA: 56s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 55s - loss: 0.69 - ETA: 55s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 54s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 53s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 52s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 51s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 50s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 49s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 48s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.69 - ETA: 47s - loss: 0.

In [43]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.0428, Recall: 1.0000, F1-measure: 0.0822
For threshold:  0.2
Micro-average quality numbers
Precision: 0.0428, Recall: 1.0000, F1-measure: 0.0822
For threshold:  0.3
Micro-average quality numbers
Precision: 0.0428, Recall: 1.0000, F1-measure: 0.0822
For threshold:  0.4
Micro-average quality numbers
Precision: 0.0428, Recall: 1.0000, F1-measure: 0.0822
For threshold:  0.5
Micro-average quality numbers
Precision: 0.0428, Recall: 1.0000, F1-measure: 0.0822
For threshold:  0.6


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.7


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [44]:
# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-3.h5', save_best_only=True)
# ]
# history = model.fit(padded_docs_train, y_train,
#                     class_weight='balanced',
#                     epochs=5,
#                     batch_size=32,
#                     validation_split=0.1,
#                     callbacks=callbacks)

In [45]:
# filter_length = 300
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Dropout(0.5))
model.add(Conv1D(300, 3, activation='relu'))
model.add(GlobalMaxPool1D())
# model.add(Conv1D(200, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
model.add(Dense(71))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1200, 71)          7013522   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1200, 71)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1198, 300)         64200     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 71)                21371     
_________________________________________________________________
activation_2 (Activation)    (None, 71)                0         
Total params: 7,099,093
Trainable params: 7,099,093
Non-trainable params: 0
_________________________________________________________________


In [46]:
# model.fit(x_train, y_train, batch_size=16, epochs=10)
# log_path=str('C:\Users\Cyborg\Documents\GitHub\Movie-Plot-Synopses')
# model_saver =  ModelCheckpoint(log_path + "/model.ckpt.{epoch:04d}.hdf5", monitor='loss', period=1)
# callbacks = [tensorboard, TerminateOnNaN(), model_saver]


model.fit(padded_docs_train, y_train,
          class_weight='balanced',
          epochs=5,
          batch_size=32,
          validation_split=0.2)
predictions=model.predict(padded_docs_test)

Train on 7591 samples, validate on 1898 samples
Epoch 1/5
7591/7591 [==============================] - ETA: 8:41 - loss: 0.703 - ETA: 6:06 - loss: 0.698 - ETA: 5:12 - loss: 0.692 - ETA: 4:44 - loss: 0.686 - ETA: 4:28 - loss: 0.679 - ETA: 4:17 - loss: 0.673 - ETA: 4:08 - loss: 0.666 - ETA: 4:02 - loss: 0.659 - ETA: 3:57 - loss: 0.651 - ETA: 3:52 - loss: 0.644 - ETA: 3:49 - loss: 0.636 - ETA: 3:45 - loss: 0.628 - ETA: 3:42 - loss: 0.621 - ETA: 3:40 - loss: 0.612 - ETA: 3:38 - loss: 0.603 - ETA: 3:35 - loss: 0.595 - ETA: 3:33 - loss: 0.585 - ETA: 3:32 - loss: 0.577 - ETA: 3:31 - loss: 0.567 - ETA: 3:29 - loss: 0.557 - ETA: 3:27 - loss: 0.547 - ETA: 3:26 - loss: 0.537 - ETA: 3:25 - loss: 0.526 - ETA: 3:25 - loss: 0.516 - ETA: 3:23 - loss: 0.506 - ETA: 3:21 - loss: 0.496 - ETA: 3:20 - loss: 0.487 - ETA: 3:18 - loss: 0.477 - ETA: 3:17 - loss: 0.467 - ETA: 3:16 - loss: 0.457 - ETA: 3:14 - loss: 0.448 - ETA: 3:13 - loss: 0.440 - ETA: 3:12 - loss: 0.432 - ETA: 3:10 - loss: 0.424 - ETA: 3:09 - l

7591/7591 [==============================] - ETA: 3:23 - loss: 0.111 - ETA: 3:22 - loss: 0.120 - ETA: 3:22 - loss: 0.119 - ETA: 3:21 - loss: 0.123 - ETA: 3:20 - loss: 0.131 - ETA: 3:19 - loss: 0.139 - ETA: 3:19 - loss: 0.135 - ETA: 3:18 - loss: 0.141 - ETA: 3:17 - loss: 0.143 - ETA: 3:16 - loss: 0.142 - ETA: 3:15 - loss: 0.140 - ETA: 3:14 - loss: 0.141 - ETA: 3:14 - loss: 0.143 - ETA: 3:13 - loss: 0.142 - ETA: 3:13 - loss: 0.141 - ETA: 3:12 - loss: 0.142 - ETA: 3:11 - loss: 0.142 - ETA: 3:10 - loss: 0.144 - ETA: 3:09 - loss: 0.143 - ETA: 3:08 - loss: 0.142 - ETA: 3:08 - loss: 0.142 - ETA: 3:07 - loss: 0.142 - ETA: 3:06 - loss: 0.142 - ETA: 3:05 - loss: 0.143 - ETA: 3:04 - loss: 0.143 - ETA: 3:03 - loss: 0.143 - ETA: 3:02 - loss: 0.143 - ETA: 3:01 - loss: 0.144 - ETA: 3:00 - loss: 0.143 - ETA: 3:00 - loss: 0.143 - ETA: 2:59 - loss: 0.143 - ETA: 2:58 - loss: 0.143 - ETA: 2:57 - loss: 0.143 - ETA: 2:56 - loss: 0.143 - ETA: 2:55 - loss: 0.143 - ETA: 2:54 - loss: 0.143 - ETA: 2:53 - loss: 0

7591/7591 [==============================] - ETA: 3:24 - loss: 0.126 - ETA: 3:25 - loss: 0.127 - ETA: 3:24 - loss: 0.125 - ETA: 3:23 - loss: 0.134 - ETA: 3:23 - loss: 0.139 - ETA: 3:22 - loss: 0.137 - ETA: 3:21 - loss: 0.138 - ETA: 3:21 - loss: 0.138 - ETA: 3:20 - loss: 0.139 - ETA: 3:18 - loss: 0.140 - ETA: 3:18 - loss: 0.140 - ETA: 3:16 - loss: 0.140 - ETA: 3:16 - loss: 0.141 - ETA: 3:15 - loss: 0.141 - ETA: 3:14 - loss: 0.142 - ETA: 3:13 - loss: 0.144 - ETA: 3:11 - loss: 0.142 - ETA: 3:10 - loss: 0.143 - ETA: 3:09 - loss: 0.143 - ETA: 3:09 - loss: 0.143 - ETA: 3:08 - loss: 0.143 - ETA: 3:07 - loss: 0.144 - ETA: 3:06 - loss: 0.144 - ETA: 3:05 - loss: 0.144 - ETA: 3:04 - loss: 0.144 - ETA: 3:03 - loss: 0.143 - ETA: 3:02 - loss: 0.142 - ETA: 3:01 - loss: 0.142 - ETA: 3:00 - loss: 0.142 - ETA: 2:59 - loss: 0.141 - ETA: 2:58 - loss: 0.141 - ETA: 2:57 - loss: 0.140 - ETA: 2:56 - loss: 0.140 - ETA: 2:56 - loss: 0.139 - ETA: 2:55 - loss: 0.139 - ETA: 2:54 - loss: 0.139 - ETA: 2:53 - loss: 0

In [47]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.1251, Recall: 0.7809, F1-measure: 0.2156
For threshold:  0.2
Micro-average quality numbers
Precision: 0.2226, Recall: 0.5653, F1-measure: 0.3194
For threshold:  0.3
Micro-average quality numbers
Precision: 0.3160, Recall: 0.4000, F1-measure: 0.3531
For threshold:  0.4
Micro-average quality numbers
Precision: 0.4171, Recall: 0.2792, F1-measure: 0.3345
For threshold:  0.5
Micro-average quality numbers
Precision: 0.5096, Recall: 0.2007, F1-measure: 0.2880
For threshold:  0.6
Micro-average quality numbers
Precision: 0.5935, Recall: 0.1492, F1-measure: 0.2384
For threshold:  0.7
Micro-average quality numbers
Precision: 0.6544, Recall: 0.1081, F1-measure: 0.1855
For threshold:  0.8
Micro-average quality numbers
Precision: 0.7083, Recall: 0.0673, F1-measure: 0.1229
For threshold:  0.9
Micro-average quality numbers
Precision: 0.7346, Recall: 0.0252, F1-measure: 0.0487


In [33]:
n_classes=71

In [49]:
# score = model.evaluate(x_test, y_test, batch_size=16)

In [50]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# Configuring the parameters
# model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(LSTM(128))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()
# pred=[]
# x= model.predict(y_test)
# for i in x:
#     pred.append(i[:])  
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1200, 50)          4939100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 71)                9159      
Total params: 5,039,907
Trainable params: 5,039,907
Non-trainable params: 0
_________________________________________________________________


In [51]:
# model = Sequential()
# # model.add(Embedding(max_words, 20, input_length=maxlen))
# model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(LSTM(128))
# model.add(Dropout(0.15))
# model.add(GlobalMaxPool1D())
# model.add(Dense(71, activation='sigmoid'))

# # model.summary()


# model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

# callbacks = [
#     ReduceLROnPlateau(),
#     EarlyStopping(patience=4),
#     ModelCheckpoint(filepath='model-2.h5', save_best_only=True)
# ]
history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1200, 50)          4939100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 71)                9159      
Total params: 5,039,907
Trainable params: 5,039,907
Non-trainable params: 0
_________________________________________________________________
Train on 8540 samples, validate on 949 samples
Epoch 1/5
8540/8540 [==============================] - ETA: 15:00 - loss: 0.69 - ETA: 11:01 - loss: 0.69 - ETA: 9:44 - loss: 0.6914 - ETA: 9:02 - loss: 0.689 - ETA: 8:38 - loss: 0.688 - ETA: 8:21 - loss: 0.686 - ETA: 

8540/8540 [==============================] - ETA: 6:20 - loss: 0.168 - ETA: 6:13 - loss: 0.159 - ETA: 6:13 - loss: 0.152 - ETA: 6:12 - loss: 0.149 - ETA: 6:10 - loss: 0.148 - ETA: 6:08 - loss: 0.150 - ETA: 6:06 - loss: 0.146 - ETA: 6:04 - loss: 0.144 - ETA: 6:02 - loss: 0.142 - ETA: 6:01 - loss: 0.142 - ETA: 5:59 - loss: 0.145 - ETA: 5:58 - loss: 0.146 - ETA: 5:57 - loss: 0.146 - ETA: 5:55 - loss: 0.148 - ETA: 5:54 - loss: 0.148 - ETA: 5:53 - loss: 0.147 - ETA: 5:51 - loss: 0.146 - ETA: 5:50 - loss: 0.147 - ETA: 5:49 - loss: 0.147 - ETA: 5:47 - loss: 0.148 - ETA: 5:46 - loss: 0.149 - ETA: 5:44 - loss: 0.148 - ETA: 5:43 - loss: 0.149 - ETA: 5:41 - loss: 0.149 - ETA: 5:40 - loss: 0.149 - ETA: 5:38 - loss: 0.148 - ETA: 5:37 - loss: 0.149 - ETA: 5:36 - loss: 0.149 - ETA: 5:34 - loss: 0.149 - ETA: 5:33 - loss: 0.149 - ETA: 5:31 - loss: 0.150 - ETA: 5:30 - loss: 0.150 - ETA: 5:29 - loss: 0.150 - ETA: 5:27 - loss: 0.150 - ETA: 5:26 - loss: 0.149 - ETA: 5:24 - loss: 0.149 - ETA: 5:23 - loss: 0

8540/8540 [==============================] - ETA: 6:12 - loss: 0.105 - ETA: 6:24 - loss: 0.121 - ETA: 6:18 - loss: 0.130 - ETA: 6:13 - loss: 0.133 - ETA: 6:12 - loss: 0.134 - ETA: 6:10 - loss: 0.135 - ETA: 6:09 - loss: 0.137 - ETA: 6:07 - loss: 0.136 - ETA: 6:06 - loss: 0.135 - ETA: 6:04 - loss: 0.136 - ETA: 6:02 - loss: 0.138 - ETA: 6:01 - loss: 0.135 - ETA: 5:59 - loss: 0.135 - ETA: 5:58 - loss: 0.136 - ETA: 5:56 - loss: 0.136 - ETA: 5:54 - loss: 0.137 - ETA: 5:53 - loss: 0.139 - ETA: 5:51 - loss: 0.140 - ETA: 5:50 - loss: 0.140 - ETA: 5:48 - loss: 0.141 - ETA: 5:47 - loss: 0.143 - ETA: 5:45 - loss: 0.142 - ETA: 5:43 - loss: 0.143 - ETA: 5:42 - loss: 0.143 - ETA: 5:41 - loss: 0.143 - ETA: 5:39 - loss: 0.144 - ETA: 5:37 - loss: 0.146 - ETA: 5:36 - loss: 0.147 - ETA: 5:35 - loss: 0.147 - ETA: 5:33 - loss: 0.148 - ETA: 5:31 - loss: 0.147 - ETA: 5:30 - loss: 0.146 - ETA: 5:28 - loss: 0.146 - ETA: 5:27 - loss: 0.146 - ETA: 5:26 - loss: 0.147 - ETA: 5:24 - loss: 0.147 - ETA: 5:23 - loss: 0

In [52]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.2114, Recall: 0.5668, F1-measure: 0.3080
For threshold:  0.2
Micro-average quality numbers
Precision: 0.2917, Recall: 0.3030, F1-measure: 0.2973
For threshold:  0.3
Micro-average quality numbers
Precision: 0.3903, Recall: 0.1367, F1-measure: 0.2024
For threshold:  0.4
Micro-average quality numbers
Precision: 0.4897, Recall: 0.0079, F1-measure: 0.0155
For threshold:  0.5


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.6


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.7


...

Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [54]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# Configuring the parameters
# model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(LSTM(128))
model.add(LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1200, 50)          4939100   
_________________________________________________________________
lstm_4 (LSTM)                (None, 1200, 128)         91648     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1200, 128)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 71)                4615      
Total params: 5,084,771
Trainable params: 5,084,771
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.compile(optimizer='adam', loss='binary_crossentropy')

history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

Train on 8540 samples, validate on 949 samples
Epoch 1/5
8540/8540 [==============================] - ETA: 23:30 - loss: 0.69 - ETA: 17:48 - loss: 0.69 - ETA: 16:00 - loss: 0.69 - ETA: 14:58 - loss: 0.68 - ETA: 14:21 - loss: 0.68 - ETA: 13:55 - loss: 0.68 - ETA: 13:34 - loss: 0.68 - ETA: 13:20 - loss: 0.67 - ETA: 13:10 - loss: 0.67 - ETA: 13:02 - loss: 0.66 - ETA: 12:46 - loss: 0.65 - ETA: 12:37 - loss: 0.64 - ETA: 12:26 - loss: 0.63 - ETA: 12:18 - loss: 0.62 - ETA: 12:10 - loss: 0.61 - ETA: 12:03 - loss: 0.60 - ETA: 11:55 - loss: 0.59 - ETA: 11:49 - loss: 0.58 - ETA: 11:45 - loss: 0.57 - ETA: 11:38 - loss: 0.56 - ETA: 11:35 - loss: 0.56 - ETA: 11:31 - loss: 0.55 - ETA: 11:26 - loss: 0.54 - ETA: 11:20 - loss: 0.53 - ETA: 11:16 - loss: 0.52 - ETA: 11:10 - loss: 0.51 - ETA: 11:05 - loss: 0.50 - ETA: 11:01 - loss: 0.50 - ETA: 10:56 - loss: 0.49 - ETA: 10:51 - loss: 0.48 - ETA: 10:46 - loss: 0.48 - ETA: 10:41 - loss: 0.47 - ETA: 10:37 - loss: 0.46 - ETA: 10:32 - loss: 0.46 - ETA: 10:28 - l

8540/8540 [==============================] - ETA: 14:50 - loss: 0.14 - ETA: 15:15 - loss: 0.14 - ETA: 15:29 - loss: 0.14 - ETA: 15:33 - loss: 0.14 - ETA: 15:53 - loss: 0.15 - ETA: 16:00 - loss: 0.15 - ETA: 15:53 - loss: 0.15 - ETA: 15:59 - loss: 0.15 - ETA: 15:59 - loss: 0.15 - ETA: 15:48 - loss: 0.15 - ETA: 15:37 - loss: 0.15 - ETA: 15:32 - loss: 0.15 - ETA: 15:22 - loss: 0.15 - ETA: 15:13 - loss: 0.15 - ETA: 15:06 - loss: 0.15 - ETA: 15:03 - loss: 0.15 - ETA: 15:03 - loss: 0.15 - ETA: 15:05 - loss: 0.15 - ETA: 14:57 - loss: 0.15 - ETA: 14:49 - loss: 0.15 - ETA: 14:38 - loss: 0.15 - ETA: 14:28 - loss: 0.15 - ETA: 14:17 - loss: 0.15 - ETA: 14:08 - loss: 0.15 - ETA: 14:01 - loss: 0.15 - ETA: 13:51 - loss: 0.14 - ETA: 13:43 - loss: 0.14 - ETA: 13:35 - loss: 0.14 - ETA: 13:28 - loss: 0.14 - ETA: 13:20 - loss: 0.14 - ETA: 13:14 - loss: 0.15 - ETA: 13:07 - loss: 0.15 - ETA: 13:01 - loss: 0.15 - ETA: 12:56 - loss: 0.15 - ETA: 12:51 - loss: 0.15 - ETA: 12:43 - loss: 0.15 - ETA: 12:35 - loss: 

8540/8540 [==============================] - ETA: 11:28 - loss: 0.14 - ETA: 11:19 - loss: 0.15 - ETA: 11:11 - loss: 0.15 - ETA: 11:07 - loss: 0.15 - ETA: 11:04 - loss: 0.15 - ETA: 11:01 - loss: 0.15 - ETA: 10:57 - loss: 0.15 - ETA: 10:55 - loss: 0.14 - ETA: 10:52 - loss: 0.15 - ETA: 10:49 - loss: 0.14 - ETA: 10:47 - loss: 0.15 - ETA: 10:44 - loss: 0.15 - ETA: 10:41 - loss: 0.15 - ETA: 10:38 - loss: 0.15 - ETA: 10:35 - loss: 0.15 - ETA: 10:33 - loss: 0.15 - ETA: 10:30 - loss: 0.14 - ETA: 10:27 - loss: 0.14 - ETA: 10:25 - loss: 0.14 - ETA: 10:22 - loss: 0.14 - ETA: 10:19 - loss: 0.14 - ETA: 10:16 - loss: 0.14 - ETA: 10:14 - loss: 0.14 - ETA: 10:11 - loss: 0.14 - ETA: 10:09 - loss: 0.14 - ETA: 10:06 - loss: 0.14 - ETA: 10:04 - loss: 0.14 - ETA: 10:02 - loss: 0.14 - ETA: 9:59 - loss: 0.1437 - ETA: 9:57 - loss: 0.144 - ETA: 9:54 - loss: 0.145 - ETA: 9:52 - loss: 0.144 - ETA: 9:52 - loss: 0.144 - ETA: 9:49 - loss: 0.145 - ETA: 9:46 - loss: 0.145 - ETA: 9:44 - loss: 0.145 - ETA: 9:41 - loss: 

In [57]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.2110, Recall: 0.5624, F1-measure: 0.3069
For threshold:  0.2
Micro-average quality numbers
Precision: 0.2842, Recall: 0.2910, F1-measure: 0.2875
For threshold:  0.3
Micro-average quality numbers
Precision: 0.3304, Recall: 0.2299, F1-measure: 0.2711
For threshold:  0.4
Micro-average quality numbers
Precision: 0.0338, Recall: 0.0013, F1-measure: 0.0026
For threshold:  0.5
Micro-average quality numbers
Precision: 0.0148, Recall: 0.0004, F1-measure: 0.0009
For threshold:  0.6


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.7


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [72]:
input_dim = (128,)     

checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')

classifier = Sequential()

#convolutional layer 1
# classifier.add(Convolution1D(32, kernel_size=3,input_shape = 1200, activation = 'relu'))
model.add(Embedding(vocab_size, 71, input_length=1200))
# classifier.add(MaxPooling1D(pool_size=2)) 
classifier.add(Dropout(.20))

#convolutional layer 2
classifier.add(Convolution1D(32, kernel_size=3, activation = 'sigmoid'))
classifier.add(MaxPooling1D(pool_size=2))
classifier.add(Dropout(.20))

# classifier.add(Convolution1D(32, kernel_size=3, activation = 'sigmoid'))
# classifier.add(MaxPooling1D(pool_size=2))
# classifier.add(Dropout(.20))

#flatten 
classifier.add(Flatten())
classifier.add(Dense(output_dim =  71, activation='sigmoid'))

C:\Users\Cyborg\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning:

Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=71)`



In [74]:
# classifier.compile(optimizer='adam', loss='binary_crossentropy')

# history = classifier.fit(padded_docs_train, y_train,
#                     class_weight='balanced',
#                     epochs=5,
#                     batch_size=32,
#                     validation_split=0.1)

In [107]:
from keras import layers

model = Sequential()
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(layers.Embedding(vocab_size, embedding_dim,  
#                            input_length=1200, 
#                            trainable=True))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(100, activation='sigmoid'))
# classifier.add(Dropout(.70))

model.add(layers.Dense(71, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy')
model.summary()

# model.compile(optimizer='adam', loss='binary_crossentropy')

model.compile(loss='binary_crossentropy', 
                  optimizer='adam')

# history = model.fit(padded_docs_train, y_train,
#                     class_weight='balanced',
#                     epochs=5,
#                     batch_size=32,
#                     validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1200, 50)          6109750   
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_4 (Dense)              (None, 71)                7171      
Total params: 6,122,021
Trainable params: 6,122,021
Non-trainable params: 0
_________________________________________________________________


In [110]:
model.fit(padded_docs_train, y_train,
                        epochs=10,
                        verbose=False,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)

In [113]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.2298, Recall: 0.4838, F1-measure: 0.3116
For threshold:  0.2
Micro-average quality numbers
Precision: 0.3091, Recall: 0.3638, F1-measure: 0.3342
For threshold:  0.3
Micro-average quality numbers
Precision: 0.3663, Recall: 0.2902, F1-measure: 0.3238
For threshold:  0.4
Micro-average quality numbers
Precision: 0.4177, Recall: 0.2345, F1-measure: 0.3004
For threshold:  0.5
Micro-average quality numbers
Precision: 0.4738, Recall: 0.1933, F1-measure: 0.2746
For threshold:  0.6
Micro-average quality numbers
Precision: 0.5327, Recall: 0.1572, F1-measure: 0.2427
For threshold:  0.7
Micro-average quality numbers
Precision: 0.5785, Recall: 0.1246, F1-measure: 0.2050
For threshold:  0.8
Micro-average quality numbers
Precision: 0.6372, Recall: 0.0927, F1-measure: 0.1618
For threshold:  0.9
Micro-average quality numbers
Precision: 0.6852, Recall: 0.0601, F1-measure: 0.1105


In [114]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(layers.Embedding(vocab_size, embedding_dim,  
#                            input_length=1200, 
#                            trainable=True))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(100, activation='sigmoid'))
model.add(Dropout(.70))
model.add(layers.Dense(100, activation='sigmoid'))
model.add(layers.Dense(71, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1200, 50)          6109750   
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               5100      
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 71)                7171      
Total params: 6,132,121
Trainable params: 6,132,121
Non-trainable params: 0
_________________________________________________________________


In [115]:
model.fit(padded_docs_train, y_train,
                        epochs=10,
                        verbose=False,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)

In [116]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.2299, Recall: 0.5640, F1-measure: 0.3266
For threshold:  0.2
Micro-average quality numbers
Precision: 0.3369, Recall: 0.3674, F1-measure: 0.3515
For threshold:  0.3
Micro-average quality numbers
Precision: 0.4548, Recall: 0.2483, F1-measure: 0.3212
For threshold:  0.4
Micro-average quality numbers
Precision: 0.5458, Recall: 0.1745, F1-measure: 0.2644
For threshold:  0.5
Micro-average quality numbers
Precision: 0.6115, Recall: 0.1343, F1-measure: 0.2203
For threshold:  0.6
Micro-average quality numbers
Precision: 0.6531, Recall: 0.0927, F1-measure: 0.1623
For threshold:  0.7
Micro-average quality numbers
Precision: 0.7085, Recall: 0.0488, F1-measure: 0.0913
For threshold:  0.8
Micro-average quality numbers
Precision: 0.7125, Recall: 0.0126, F1-measure: 0.0248
For threshold:  0.9
Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [ ]:
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Dropout(0.5))
model.add(Conv1D(300, 3, activation='relu'))
model.add(GlobalMaxPool1D())
# model.add(Conv1D(200, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
model.add(Dense(71))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

In [119]:
# checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')
model = Sequential()
# model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Conv1D(64, 3, activation='sigmoid'))
# model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Conv1D(48, 3, activation='sigmoid'))
model.add(Flatten())
# model.add(Dense(6, activation='softmax'))
model.add(Dense(71))
# model.add(Dropout(0.70))
model.compile(optimizer='adam', loss='binary_crossentropy')
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1200, 71)          8675845   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1198, 64)          13696     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1196, 100)         19300     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1194, 100)         30100     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1192, 48)          14448     
_________________________________________________________________
flatten_2 (Flatten)          (None, 57216)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 71)                4062407   
Total para

In [120]:
model.fit(padded_docs_train, y_train,
                        epochs=10,
                        verbose=False,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)

In [121]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279
For threshold:  0.2
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279
For threshold:  0.3
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279
For threshold:  0.4
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279
For threshold:  0.5
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279
For threshold:  0.6
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279
For threshold:  0.7
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279
For threshold:  0.8
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279
For threshold:  0.9
Micro-average quality numbers
Precision: 0.1717, Recall: 0.3387, F1-measure: 0.2279


In [123]:
model = Sequential()
# model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Conv1D(64, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='sigmoid'))
model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='sigmoid'))
model.add(Flatten())
# model.add(Dense(6, activation='softmax'))
model.add(Dense(71))

model.compile(optimizer='adam', loss='binary_crossentropy')
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 1200, 71)          8675845   
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 1198, 64)          13696     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1198, 64)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 1196, 100)         19300     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1196, 100)         0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 1194, 100)         30100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 1194, 100)         0         
__________

In [124]:
model.fit(padded_docs_train, y_train,
                        epochs=10,
                        verbose=False,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)


predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499
For threshold:  0.2
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499
For threshold:  0.3
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499
For threshold:  0.4
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499
For threshold:  0.5
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499
For threshold:  0.6
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499
For threshold:  0.7
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499
For threshold:  0.8
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499
For threshold:  0.9
Micro-average quality numbers
Precision: 0.2200, Recall: 0.2893, F1-measure: 0.2499


In [37]:
model = Sequential()
# Configuring the parameters
# model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
# model.add(LSTM(128))
model.add(LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(71, activation='relu'))
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1200, 50)          6109750   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1200, 128)         91648     
_________________________________________________________________
dropout_4 (Dropout)          (None, 1200, 128)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 71)                4615      
Total params: 6,255,421
Trainable params: 6,255,421
Non-trainable par

In [39]:

model.compile(optimizer='adam', loss='binary_crossentropy')

model.fit(padded_docs_train, y_train,
                        epochs=10,
                        verbose=False,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)


predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.2137, Recall: 0.5620, F1-measure: 0.3096
For threshold:  0.2
Micro-average quality numbers
Precision: 0.3483, Recall: 0.2290, F1-measure: 0.2763
For threshold:  0.3
Micro-average quality numbers
Precision: 0.3894, Recall: 0.1280, F1-measure: 0.1927
For threshold:  0.4


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.5


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.6


...

Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000


In [34]:
model = Sequential()
# model.add(Conv1D(180, 3, input_shape=(timesteps, input_dim), activation='relu'))
model.add(Embedding(vocab_size, 71, input_length=1200))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(100, 3, activation='relu'))
model.add(Dropout(0.70))
model.add(Conv1D(48, 3, activation='relu'))
model.add(Flatten())
# model.add(Dense(6, activation='softmax'))
model.add(Dense(71))

model.compile(optimizer='adam', loss='binary_crossentropy')
# model.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy']) 
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1200, 71)          8675845   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1198, 64)          13696     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1198, 64)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1196, 100)         19300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1196, 100)         0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1194, 100)         30100  

In [35]:
# model.compile(optimizer='adam', loss='binary_crossentropy')

model.fit(padded_docs_train, y_train,
                        epochs=10,
                        verbose=False,
                        validation_data=(padded_docs_test, y_test),
                        batch_size=16)




In [36]:
predictions=model.predict([padded_docs_test])
for val in thresholds:
    print("For threshold: ", val)
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

For threshold:  0.1
Micro-average quality numbers
Precision: 0.0849, Recall: 0.0824, F1-measure: 0.0836
For threshold:  0.2


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.3


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.4


...

Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.8


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
For threshold:  0.9


C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples.

C:\Users\Cyborg\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no predicted samples.



Micro-average quality numbers
Precision: 0.0000, Recall: 0.0000, F1-measure: 0.0000
